In [ ]:
!pip -q install huggingface_hub 
!huggingface-cli download badger-lord/embeddings2025 --repo-type dataset --local-dir ./embeddings2025 --quiet

In [ ]:
!pip -q install umap-learn

In [ ]:
%%time
import pandas as pd 

embeddings_df = pd.read_parquet('embeddings2025/embeddings.parquet')
captions_df   = pd.read_parquet('embeddings2025/captions.parquet')

In [ ]:
X_embeddings=embeddings_df.values
y_captions  = captions_df.values
X_embeddings.shape

In [ ]:
%%time

from sklearn.cluster import KMeans
# 1. Instantiate the model
kmeans = KMeans(n_clusters=10, random_state=0, n_init='auto')

# 2. Fit and predict cluster labels
kmeans_labels = kmeans.fit_predict(X_embeddings)  

# 3. Inspect results
kmeans_centroids = kmeans.cluster_centers_
print("Labels:", kmeans_labels)
print("Centroids:", kmeans_centroids)

# 10  clusters took 53.2seconds on Brev: data is (652671 x 768d)
# 300 clusters took 21minutes 9 seconds on Brev. same data

In [ ]:
%%time
from sklearn.decomposition import PCA

# 1a. If X is a pandas DataFrame, convert to NumPy
X_np = X_embeddings.values if hasattr(X_embeddings, "values") else X_embeddings

# 1b. PCA to 50 components
pca = PCA(n_components=10, random_state=42)
X_reduced = pca.fit_transform(X_np)

# took 3.57 seconds to do this 768d-> 10d

In [ ]:
%%time

from sklearn.cluster import KMeans
# 1. Instantiate the model
kmeans = KMeans(n_clusters=300, random_state=0, n_init='auto')

# 2. Fit and predict cluster labels
kmeans_labels_reduced = kmeans.fit_predict(X_reduced)  

# 3. Inspect results
kmeans_centroids = kmeans.cluster_centers_
print("Labels:", kmeans_labels_reduced)
print("Centroids:", kmeans_centroids)

# 300 clusters took 1min 25s with data (652671 x 10d)
# 300 clusters took 1min 51s with data (652671 x 50d)

In [ ]:
%%time

import umap  # pip install umap-learn

manifold = umap.UMAP(n_neighbors=30,min_dist=0.5).fit(X_reduced)
X_reduced_umap = manifold.transform(X_reduced)

# 2 components 30 neighbors min_dist=0.5 took 13 minutes 10s on (652671 x 50d)
# 2 components 30 neighbors min_dist=0.5 took 12 minutes 6s on (652671 x 3d )
# 2 components default umap parameters took (652671 x 3d )

In [ ]:
%%time
import umap  

manifold = umap.UMAP(n_neighbors=30,min_dist=0.5,n_components=3).fit(X_np)
X_reduced_umap = manifold.transform(X_np)

# will not run on cpu